In [1]:
# install dependencies
!pip install pandas sklearn numerapi

In [2]:
# import dependencies
import pandas as pd
import numerapi
import sklearn.linear_model

In [3]:
# download the latest tournament dataset (takes around 30s)
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")

In [4]:
# find only the feature columns
feature_cols = tournament_data.columns[tournament_data.columns.str.startswith('feature')]
feature_cols = feature_cols.tolist()

In [5]:
# predict the target on the live features
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor.load("AutogluonModels/ag-20220618_131043/")

In [6]:
df_reader = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz", chunksize=4096)

In [7]:
y_pred = []
for df_chunk in df_reader:
    y_pred.append(predictor.predict(df_chunk[feature_cols], as_pandas=True))
y_pred = pd.concat(y_pred, axis=0, ignore_index=True)

In [8]:
# predictions must have an `id` column and a `prediction` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction"] = y_pred
predictions_df.head()

,id,prediction
0,n0003aa52cab36c2,0.488228
1,n000920ed083903f,0.488633
2,n0038e640522c4a6,0.525052
3,n004ac94a87dc54b,0.496688
4,n0052fe97ea0c05f,0.506878


In [ ]:
# Upload your predictions
predictions_df.to_csv("autogluon_predictions.csv", index=False)

In [9]:
# Get your API keys and model_id from https://numer.ai/notebook
public_id = ""
secret_key = ""
model_id = ""
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [11]:
submission_id = napi.upload_predictions("autogluon_predictions.csv", model_id=model_id)

Timeout Error: HTTPSConnectionPool(host='api-tournament.numer.ai', port=443): Read timed out. (read timeout=3)


KeyError: 'data'